In [ ]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode
from google.colab.patches import cv2_imshow
import cv2
import numpy as np
import os
import pandas as pd
from datetime import datetime
import pytz
import torch
from torchvision import transforms
from torchvision.models import resnet18
from torch.nn.functional import normalize
import time
from google.colab import files

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"El dispositivo actual es: {device}")

In [ ]:
import zipfile
import os

# Especifica el nombre del archivo ZIP cargado
zip_file = "images.zip"

# Define el directorio de extracción
output_dir = "/content/images"

# Crea el directorio si no existe
os.makedirs(output_dir, exist_ok=True)

# Descomprimir el archivo
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print(f"Archivo descomprimido en {output_dir}")

In [ ]:
input_dir = "/content/images"
reference_embeddings = {}

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
def get_embedding(image):
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = model(image).squeeze(0)
    return normalize(embedding, p=2, dim=0).cpu().numpy()

In [ ]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
model = resnet18(weights='IMAGENET1K_V1')
model.fc = torch.nn.Identity()
model = model.to(device)
model.eval()

In [ ]:
attendance_log = []

In [ ]:
for person_dir in os.listdir(input_dir):
    person_path = os.path.join(input_dir, person_dir)
    if os.path.isdir(person_path):
        for filename in os.listdir(person_path):
            if filename.endswith(".jpg"):
                image = cv2.imread(os.path.join(person_path, filename))
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
                if len(faces) > 0:
                    x, y, w, h = faces[0]
                    face = image[y:y + h, x:x + w]
                    embedding = get_embedding(face)
                    reference_embeddings[person_dir] = embedding

In [ ]:
def js_to_image(js_reply):
    image_bytes = b64decode(js_reply.split(',')[1])
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    img = cv2.imdecode(jpg_as_np, flags=1)
    return img

In [ ]:
def take_photo(quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Fotito';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)

    data = eval_js('takePhoto({})'.format(quality))
    img = js_to_image(data)

    timestamp = time.strftime("%Y%m%d_%H%M%S")
    filename = f'photo_{timestamp}.jpg'
    cv2.imwrite(filename, img)

    return filename

In [ ]:
def recognize_and_log():
    filename = take_photo()
    image = cv2.imread(filename)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    local_tz = pytz.timezone('America/Lima')

    for (x, y, w, h) in faces:
        face = image[y:y + h, x:x + w]
        embedding = get_embedding(face)

        min_distance = float("inf")
        recognized_name = "Desconocido"
        for name, ref_embedding in reference_embeddings.items():
            distance = np.linalg.norm(embedding - ref_embedding)
            if distance < min_distance and distance < 0.6:
                min_distance = distance
                recognized_name = name
                local_now = datetime.now(local_tz).strftime("%Y-%m-%d %H:%M:%S")

        if recognized_name != "Desconocido":
            # Registrar información de asistencia
            attendance_log.append({
                "Nombre": recognized_name,
                "Fecha de entrada": local_now,
                "Imagen": filename
            })
            print(f"Reconocido: {recognized_name}")

            # Dibujar un rectángulo alrededor del rostro reconocido
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(image, recognized_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        else:
            print("Rostro no reconocido.")
            # Dibujar un rectángulo en rojo para rostros no reconocidos
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(image, "Desconocido", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    # Mostrar la imagen con las anotaciones utilizando cv2_imshow
    cv2_imshow(image)

In [ ]:
def automated_recognition():
    while True:
        recognize_and_log()
        print("Captura realizada. Esperando 10 segundos para la siguiente.")
        time.sleep(10)  # Espera 10 segundos
        if input("¿Deseas continuar? (s/n): ").lower() != 's':
            break


In [ ]:
automated_recognition()

In [ ]:
df = pd.DataFrame(attendance_log)
df.to_csv("attendance_log.csv", index=False)
print("Registro de asistencia guardado en 'attendance_log.csv'")

In [ ]:
files.download("attendance_log.csv")